In [11]:
from extract import extract
from extract_all import extract_all
import PyPDF2 as p
import config as conf

In [2]:
def longest_seq(li: list):
    longest = []
    seq = [li[0]]
    longest_streak = 0
    for i in range(len(li)-1):
        if li[i] == li[i+1]-1:
            seq = seq + [li[i+1]]
            streak = streak + 1
        else:
            streak = 0
            seq = [li[i+1]]
        if longest_streak < streak:
            longest_streak = streak
            longest = seq
    return longest

In [3]:
def get_pg_rng(pdf_url):
    pdf = p.PdfFileReader(pdf_url)
    n = pdf.numPages
    outp = ''
    pgs = list()
    for pg in range(n):
        page = pdf.getPage(pg)
        content = page.extractText()
        if all([target in content.lower() for target in conf.TARGET_SENTENCE]):
            pgs = pgs + [pg]
    if len(pgs) > 0:
        pgs = longest_seq(pgs)
        return str(min(pgs)) + '-' + str(max(pgs))
    else:
        raise(BaseException('No pages contain tables: '+pdf_url))

In [4]:
longest_seq([5,10,11,12,50,55,56,57,58,59])

[55, 56, 57, 58, 59]

In [12]:
df = extract('../test_data/pdf/01-IGACOS2010_Audit_Report.pdf')

                       Audit\rObservations (CY 2009)  \
0  Non-settingupof\rRPT/SET Receivables\r1.RealPr...   
1  Unrecorded\rInterest Income\r\r2.   Failure to...   
2                                                NaN   
3                      Audit\rObservations (CY 2009)   
4  Understated Insurance\rExpense\r.    The expir...   
5  Uncancelled Stale\rChecks\r4.Stalechecks\ramou...   

                                     Recommendations  Ref  Management\rAction  \
0  DirecttheCity\rTreasurer  to  submit\raCertifi...  NaN         Implemented   
1  DirecttheCity\rAccountant  to  take\rupthenece...  NaN         Implemented   
2                                        statements.  NaN                 NaN   
3                                    Recommendations  Ref  Management\rAction   
4  DirecttheCity\rAccountantto\rreview all\rprepa...  NaN         Implemented   
5  DirecttheCity\rAccountantto\rregularly  review...  NaN         Implemented   

   Status of\rImplementat\rion  Reason 

https://unix.stackexchange.com/questions/533005/moving-a-file-based-on-specific-filename-to-specific-folder-based-on-filename

In [14]:
df

""


In [7]:
df[0]

KeyError: 0